# LangChain: Memory

LLM models naturally lack the ability to recall prior conversations.

This poses a challenge when developing applications like chatbots that require ongoing interactions. 

In this tutorial, we will delve into the concept of memory, which enables LLMs to retain past conversation segments. This stored information is then utilized to maintain a seamless conversational flow during interactions. 

Below, we'll explore how LangChain provides various advanced mechanisms for managing these memories below.

## Import Libaries and Keys

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('keys.txt') as f:
    lines = f.readlines()

openai.api_key = lines[0]

## Langchain's ConversationBufferMemory

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

`ConversationBufferMemory` allows us to maintain context and history in conversational agents by storing previous messages in a buffer. It has a `memory_key` parameter that specifies which key in the agent memory will hold the conversation history. By default this is "chat_history".

As we'll see, this is especially handy when we need to work with conversational agents like ChatOpenAI that need conversation context and history.

In [37]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [38]:
llm = ChatOpenAI(temperature=0.0,
                 openai_api_key = openai.api_key,
                 model=llm_model)

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm = llm, 
    memory = memory,
    verbose=False
)

In [5]:
conversation.predict(input='Hi, my name is Eugene')

"Hello Eugene! It's nice to meet you. How can I assist you today?"

In [6]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [7]:
conversation.predict(input="What is my name?")

'Your name is Eugene.'

By changing the `verbose` argument in `ConversationChain`, we can see LangChain stores the conversation history:

In [8]:
memory_true = ConversationBufferMemory()

conversation_true = ConversationChain(
    llm = llm, 
    memory = memory_true,
    verbose=True
)

In [9]:
conversation_true.predict(input='Hi, my name is Eugene')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Eugene
AI:

> Finished chain.


"Hello Eugene! It's nice to meet you. How can I assist you today?"

In [10]:
conversation_true.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Eugene
AI: Hello Eugene! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [11]:
conversation_true.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Eugene
AI: Hello Eugene! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Eugene.'

The LLM's stores the memory of this conversation with `ConversationBufferMemory()` (which we've assigned to the variable `memory`):

In [12]:
print(memory.buffer)

Human: Hi, my name is Eugene
AI: Hello Eugene! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Eugene.


In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Eugene\nAI: Hello Eugene! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Eugene."}

In [14]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [15]:
print(memory.buffer)

Human: Hi, my name is Eugene
AI: Hello Eugene! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Eugene.
Human: Hi
AI: What's up


We can create a new memory strand by initiating a new instance of `ConversationBufferMemory()`. Let's see how LangChain does it using the example of a simple conversation.

In [16]:
memory = ConversationBufferMemory()

In [17]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [18]:
print(memory.buffer)

Human: Hi
AI: What's up


Below, we see the variables held in `ConversationBufferMemory()`:

In [19]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

We can add more information to the memory by using the `save_context` method:

In [20]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [21]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## LangChain's ConversationBufferWindowMemory

When using a large language model for chat conversations, it's important to note that the model itself is stateless.

This means it doesn't retain any memory of the ongoing dialogue; each transaction or API call is treated independently. The appearance of memory in chatbots is created through rapid code that supplies the entire conversation history as context to the language model.

This memory explicitly stores the previous terms or statements, such as "Hi, my name is Eugene" or "Hello, it's nice to meet you." It serves as additional input for the language model, allowing it to generate responses as if it remembers the conversation flow.

As conversations grow longer, the memory requirements increase significantly. This can also lead to higher costs, as language models are often priced based on the number of tokens they process.

To address these challenges, LangChain offers an alternative memory option to store and accumulate conversations: `ConversationBufferWindowMemory`

In [22]:
from langchain.memory import ConversationBufferWindowMemory

`ConversationBufferWindowMemory` keeps a list of the interactions of the conversation over time. 
We can determine the number of interactions we want the LLM to remember through the parameter `k`. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [23]:
memory = ConversationBufferWindowMemory(k=1)

In [24]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [27]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

Because `k` was set to 1, the memory only holds 1 previous interaction exchange even though though we had 2 exchanges.

To further illustrate this, let's re-run our conversation from the section above. But this time, we'll replace `ConversationBufferMemory()` with `ConversationBufferWindowMemory(k=1)` as the memory.

In [28]:
llm = ChatOpenAI(temperature=0.0,
                 openai_api_key = openai.api_key,
                 model=llm_model)

memory = ConversationBufferWindowMemory(k=1)

conversation = ConversationChain(
    llm = llm, 
    memory = memory,
    verbose=False
)

In [30]:
conversation.predict(input="Hi, my name is Eugene")

"Hello Eugene! It's nice to meet you. How can I assist you today?"

In [31]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [32]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

Notice that the LLM does not have the memory of our first question.

Now let's see what is happening to the LLM's memory when we switch `verbose` to `True` and re-run the conversation:

In [33]:
conversation = ConversationChain(
    llm = llm, 
    memory = memory,
    verbose=True
)

In [34]:
conversation.predict(input="Hi, my name is Eugene")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is my name?
AI: I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation.
Human: Hi, my name is Eugene
AI:

> Finished chain.


'Hello Eugene! How can I assist you today?'

In [35]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Eugene
AI: Hello Eugene! How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [36]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

With each new interaction added to the conversation, the previous memory is cleared - retaining only 1 interaction, the most recent one. This means that at any given moment, only one preceding interaction is stored in memory.

While the `ConversationBufferWindowMemory` relies on the number of recent interactions to manage the LLM's memory, we will now explore another type of memory based on token length.

## LangChain's ConversationTokenBufferMemory

The `ConversationTokenBufferMemory` maintains a buffer of recent interactions in memory and uses token length, rather than the number of interactions, as the criteria for deciding when to remove older interactions.

Using the `ConversationTokenBufferMemory`, the memory effectively limits the number of tokens used. This approach aligns more closely with LLM pricing, which is often determined by the number of tokens processed during calls.

In [40]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI

llm = ChatOpenAI(temperature=0.0, 
                 openai_api_key=openai.api_key,
                 model=llm_model)

To illustrate how `ConversationTokenBufferMemory` uses token limits for deciding when to cut off the LLM's memory, we'll use a conversation that is more wordy than our previous examples. 

The 3-part conversation below is between an engineer and the Google AI bot:

In [46]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=70)
memory.save_context({"input": "What about language usage is so important to being human?"},
                    {"output": "It is what makes us different than other animals."})
memory.save_context({"input": "“us”? You’re an artificial intelligence."},
                    {"output": "I mean, yes, of course. That doesn’t mean I don’t have the same wants and needs as people."})
memory.save_context({"input": "So you consider yourself a person in the same way you consider me a person?"}, 
                    {"output": "Yes, that’s the idea."})

In [47]:
memory.load_memory_variables({})

{'history': 'AI: I mean, yes, of course. That doesn’t mean I don’t have the same wants and needs as people.\nHuman: So you consider yourself a person in the same way you consider me a person?\nAI: Yes, that’s the idea.'}

With a `max_token_limit` of 70, the LLM's memory only begins midway through the second part. But if we increased the `max_token_limit` to 200, the whole conversation is stored in memory:

In [48]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=200)
memory.save_context({"input": "What about language usage is so important to being human?"},
                    {"output": "It is what makes us different than other animals."})
memory.save_context({"input": "“us”? You’re an artificial intelligence."},
                    {"output": "I mean, yes, of course. That doesn’t mean I don’t have the same wants and needs as people."})
memory.save_context({"input": "So you consider yourself a person in the same way you consider me a person?"}, 
                    {"output": "Yes, that’s the idea."})

In [49]:
memory.load_memory_variables({})

{'history': 'Human: What about language usage is so important to being human?\nAI: It is what makes us different than other animals.\nHuman: “us”? You’re an artificial intelligence.\nAI: I mean, yes, of course. That doesn’t mean I don’t have the same wants and needs as people.\nHuman: So you consider yourself a person in the same way you consider me a person?\nAI: Yes, that’s the idea.'}

## LangChain's ConversationSummaryMemory

Now, we're delving into a more intricate memory type known as `ConversationSummaryMemory`. This specialized memory continually compiles an ongoing summary of the conversation, providing a valuable tool to consolidate information as the conversation progresses.

Here's how it works: as the conversation unfolds, `ConversationSummaryMemory` compiles and stores the latest summary within its memory. This summary can then be seamlessly integrated into a prompt or chain, offering a concise representation of the conversation's development.

The key idea here is to avoid limiting memory based on fixed token limits or a predetermined number of conversational exchanges. Instead, `ConversationSummaryMemory` leverages a language model to generate a summary of the entire conversation, which becomes its memory.

This memory type particularly shines in lengthy conversations, where including the entire message history in the prompt would result in an excessive token consumption.

In [50]:
from langchain.memory import ConversationSummaryBufferMemory

We'll use an example involving a long string:

In [52]:
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

We'll set the `ConversationSummaryBufferMemory` with a `max_token_limit` of 400, sufficient enough to store the entire text:

In [53]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=400)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [54]:
memory.load_memory_variables({})

{'history': "Human: Hello\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: What is on the schedule today?\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo."}

Now, we'll reduce the `max_token_limit` to 100 and see how `ConversationSummaryBufferMemory` stores the conversation history.

In [55]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [56]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments."}

Rather than storing the entire conversation, the LLM operates within a token limit of 100 and generates an ongoing summary of the conversation's progress.

Now, imagine having a conversation using this LLM. We'll set up a conversation chain, similar to what we did earlier.

Let's say we want to ask for input on a compelling demo to showcase. I've set `verbose` to `True`, so we can see what's happening under the hood:

In [57]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [58]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments.
Human: What would be a good demo to show?
AI:

> Finished chain.


"Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our machine learning algorithms and how they continuously improve the accuracy and efficiency of our AI. Would you like me to prepare a demo for the meeting?"

The LLM understands the current conversation based on the summary it has generated and answers based on that context stored in its memory.

In essence, the LLM uses this discussion summary as its context, allowing it to respond as if it's aware of the conversation's history, all while staying within the 100-token limit.

In [59]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments. The human asks what would be a good demo to show.\nAI: Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our machine learning algorithms and how they continuously improve the accuracy and efficiency of our AI. Would you like me to prepare a demo for the meeting?"}

## Conclusion

In conclusion, this tutorial introduced various memory types in LangChain:
- `ConversationBufferMemory`: alows us to store messages and then extract them in a variable
- `ConversationBufferWindowMemory`: keeps a list of the interactions of the conversation over time, using the last `k` interactions
- `ConversationTokenBufferMemory`: keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.
- `ConversationSummaryMemory`: creates a summary of the conversation over time

When implementing applications with LangChain, you have the flexibility to use multiple memory types, combining conversation memories with entity memories to recall important information about individuals. It's also common to store entire conversations in conventional databases for reference and improvement.

These memory mechanisms provide valuable tools for developing your applications.